# Code Assignment 3

## Gurobi

In [1]:
using CSV
df = CSV.read("E:\\Documents\\GitHub\\mec_equil\\data\\NYC_subway\\arcs.csv", delim = ',', 
    header = true, allowmissing = :auto, rows_for_type_detect = 150)

nbNodes = maximum(df[2])
nbArcs = length(df[2])
A = spzeros(nbNodes, nbArcs)

for i in 1:length(df[1])
    A[df[1][i], i] = -1
    A[df[2][i], i] = 1
end

d = spzeros(maximum(df[2]))
d[471] = 1
d[452] = -1

c = df[3];

In [2]:
using Gurobi, JuMP
m = Model(solver=GurobiSolver())

@variable(m, π[1:nbArcs] >= 0, Int)
@constraint(m, A * π .== d)
@objective(m, Min, c'*π)

solve(m)

Academic license - for non-commercial use only
Optimize a model with 501 rows, 1290 columns and 2580 nonzeros
Variable types: 0 continuous, 1290 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 3e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 139 rows and 289 columns
Presolve time: 0.00s
Presolved: 362 rows, 1001 columns, 2002 nonzeros
Variable types: 0 continuous, 1001 integer (0 binary)
Found heuristic solution: objective 12150.936293

Root relaxation: objective 1.213381e+04, 43 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    12133.809467 12133.8095  0.00%     -    0s

Explored 0 nodes (43 simplex iterations) in 0.01 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 12133.8 12150.9 

Optimal solution found

:Optimal

In [3]:
sources = df[1][getvalue(π) .!= 0]
dests = df[2][getvalue(π) .!= 0]
union(sources, dests)

5-element Array{Int64,1}:
 452
 433
 463
 468
 471

## Bellman-Ford

In [28]:
function bellman_ford(df, c, ;max_iter = 50)
    p_new = Inf * ones(nbNodes)
    p_new[452] = 0
    p_old = zeros(nbNodes)
    iteration = 0

    while any(p_new .!= p_old)
        iteration += 1
        p_old .= p_new

        for y in 1:nbNodes
            p_new[y] = min(p_old[y], minimum(vcat(Inf, c[df[1] .== y] .+ p_old[df[df[1] .== y, 2]])))
        end
    end

    return (p_new, iteration)
end

bellman_ford (generic function with 1 method)

In [30]:
(p, iteration) = bellman_ford(df, c;max_iter = 50)

([34050.7, 33505.6, 32777.8, 32141.0, 31409.7, 30795.5, 30069.7, 29394.9, 28667.2, 27529.0  …  Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf], 24)

The shortest path has the same distance as the solution from Gurobi:

In [32]:
p[471]

12133.809467

## Regularized Problem

In [112]:
function dual_wrapper(d, A, c, T)
    return p -> d' * p - T * sum(exp.((A' * p .- c) / T))
end

dual_wrapper (generic function with 1 method)

In [113]:
# define gradient_decent()
using ForwardDiff
function gradient_decent(f, p_init; stepsize = 0.2, tol = 1e-5, max_iter = 5000)
    tic()
    iteration = 0
    residual = 1.
    p_old = p_init
    p_new = zeros(p_init)
    
    while (residual > tol) && (iteration <= max_iter)
        iteration += 1
        
        p_new .= p_old .- stepsize .* ForwardDiff.gradient(f, p_old)

        residual = maximum(abs.(p_new .- p_old))
        p_old .= p_new
    end
    
    return (residual, iteration, toq(), p_new)
end

gradient_decent (generic function with 2 methods)

In [114]:
gradient_decent(dual_wrapper(d, A, c, 1.), ones(nbNodes), 
    stepsize = 0.01, max_iter = 50)

(0.08244586096657569, 51, 0.646645119, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])